## *Visualization of a PFSS magnetic field extrapolation*
***

In this example, a PFSS magnetic field model is computed, and the result is then visualized using SHELVIS.

In [ ]:
import numpy as np

In [ ]:
import sunpy

In [ ]:
import astropy.coordinates
import astropy.units as u

In [ ]:
import sys
import shelvis

### Load magnetogram & process

In [ ]:
import cider.magnetogram
import cider.magnetogram.hmi

In [ ]:
import os
import urllib.request

sample = 'hmi.mrdailysynframe_polfil_720s.20220328_120000_TAI.Mr_polfil.fits'

if not os.path.exists('../data/'+sample):
    urllib.request.urlretrieve('https://a3s.fi/cider_sample_data/'+sample, '../data/'+sample)

In [ ]:
magnetogram_file = "../data/hmi.mrdailysynframe_polfil_720s.20220328_120000_TAI.Mr_polfil.fits"

In [ ]:
raw_magnetogram = cider.magnetogram.hmi.read_hmi_daily_synframe(magnetogram_file)

In [ ]:
import cider.utils.map

In [ ]:
# Create an empty map with the requested resolution
uniform_map \
    = cider.utils.map.create_full_sun_plate_carree_map(raw_magnetogram,
                                                       deg_per_pixel=1.0,
                                                       frame=raw_magnetogram.coordinate_frame.name)

In [ ]:
remapped_magnetogram = cider.magnetogram.regrid_to_grid_of_map(raw_magnetogram, uniform_map)

In [ ]:
balanced_magnetogram = cider.magnetogram.Balance.multiplicative(remapped_magnetogram)

### Compute the PFSS model

In [ ]:
import cider.models.pfss

In [ ]:
r = np.linspace(1.0, 2.5, 128)*astropy.constants.R_sun

In [ ]:
# Instantiate the model
pfss = cider.models.pfss.PotentialFieldSourceSurfaceModel(balanced_magnetogram, r)

In [ ]:
# Compute the solution
pfss.compute()

In [ ]:
# Compute the magnetic field
magnetic_field = pfss.magnetic_field()

In [ ]:
import cider.utils.interpolation

In [ ]:
nodal_magnetic_field \
        = cider.utils.interpolation.average_face_staggered_to_nodal(magnetic_field)

### Create a Dataset object for SHELVIS

First, the coordinate frame that the result is plotted in needs to be defined. Here, we use the Stonyhurst frame as an example.

In [ ]:
frame_of_plot = sunpy.coordinates.HeliographicStonyhurst(lon=0*u.deg, 
                                                         lat=0*u.deg, 
                                                         obstime=pfss.magnetogram.coordinate_frame.obstime)

Create the SHELVIS dataset. In doing so, the result is transformed to the requested frame

In [ ]:
import shelvis.io.cider

In [ ]:
dataset = shelvis.io.cider.Dataset(coordinate_frame=frame_of_plot)

In [ ]:
dataset.from_field(nodal_magnetic_field, frame=pfss.magnetogram.coordinate_frame)

In [ ]:
# Add the vector field
dataset.add_vector(nodal_magnetic_field, 
                   name="magnetic_field", 
                   unit=pfss.magnetogram.unit, 
                   frame=pfss.magnetogram.coordinate_frame)

In [ ]:
# Add separately the spherical vector components of the magnetic field
dataset.add_scalar(nodal_magnetic_field.data[0], name="br", unit=pfss.magnetogram.unit)
dataset.add_scalar(nodal_magnetic_field.data[1], name="bt", unit=pfss.magnetogram.unit)
dataset.add_scalar(nodal_magnetic_field.data[2], name="bp", unit=pfss.magnetogram.unit)

### Plot the result

To plot the resulting data, first the visualization widget is created. In addition to the coordinate frame, the unit of length used in defining coordinates in the plot needs to be set

In [ ]:
vis = shelvis.Visualization(coordinate_frame=frame_of_plot, unit_of_length=astropy.constants.R_sun)

Next, a spherical slice plot is added to the visualization

In [ ]:
from shelvis.widgets.implicit import Sphere

In [ ]:
vis.add(shelvis.SlicePlot(
            # The dataset to plot
            dataset,
            # The slice surface, here a sphere
            Sphere(radius=1.2,
                    min=1.001,
                    max=2.491,
                    step=0.01,
                    unit=astropy.constants.R_sun),
            # The variable to plot
            variable="br",
            # The unit to use in the plot
            unit="gauss",
            # A name to identify the plot
            name="Sphere slice (br)"))

In [ ]:
vis